In [6]:
! pip install apscheduler

  Using cached https://files.pythonhosted.org/packages/f3/34/9ef20ed473c4fd2c3df54ef77a27ae3fc7500b16b192add4720cab8b2c09/APScheduler-3.6.3-py2.py3-none-any.whl


TO DO:
- Make generic functions for tasks for modularity
- CRON jobs for all timeframes
- One function to update all csvs
- Plug n play for indicators
- Add limit to queries
- Include Active/Inactive column in CSV for active/inactive coins
- Update_CSV_to_Latest should contain active coins from Exchanges and From CSV. Check if a (coin,exchange) tuple is active(Check if it is present on exchange using ccxt library) . If it is active   get latest data for it if already present in CSV,if not in CSV get all data. If coin is not active on exchange , we will put a Active/Inactive status in CSV accordingly. All functions will have   to be modified to run code only for active coin-exchange combinations.
- Date Format will be '%d-%m-%Y %H:%M:%S' . This is giving me a lot of problems especially while reading data. When I don't put :%S it tells me dataframe has second and sometimes when :%S is         there, it tells me no second value in dataframe.
- Analytics Value Accuracy. Some parameter in Jupyter.
- Have to fetch Coins based on Parameters. Example - Fetch all active coins-exchange combinations where RSI>0 and RSI<=30. Fetch all active coins-exchange combinations where closing price is         between LOWERBAND and MIDDLEBAND. Get me intersection(common coins) of these 2 list. Now the coin from the intersection list which probably has the lowest volume can increase in price faster       then the others(Little increase in Volume will result in Big increase in Price)
- For each active coin-exchange combination I want to check the change in Value of different Technical Indicators of 2 consecutive periods in time. Example - I want to know if for a particular       coin RSI=a on period x and RSI>a on period x+1. I want to know whenever MACD and MACD_SIGNAL cross each other(On period x MACD=a and MACD_SIGNAL=b where a<=b and on period x+1 MACD=a and           MACD_SIGNAL=b where a>b. MACD_HISTOGRAM same like RSI want to know when it is 'a' on period x and 'a++' on period x+1).
- Convert 1D timeframe to 3D/1Week/etc. Convert 1H timeframe to 4H/6H/etc.
- Batch Processing of downloading new coins data

In [1]:
# Imports
import os
import time
#import talib
import logging
import requests
import datetime
import importlib
import dateutil.parser
import ccxt
import numpy as np
import pandas as pd
from importlib import reload
from collections import defaultdict
import sys
import inspect
import pyti
from apscheduler.schedulers.background import BackgroundScheduler
from IPython.core.debugger import set_trace
import pprint


In [2]:
importlib.reload(logging)
LOGGING_FORMAT = '%(asctime)s - %(levelname)s - %(message)s'
logging.basicConfig(format=LOGGING_FORMAT, level=logging.
                    INFO)


In [3]:
# FIELDS
PRICE = 'PRICE'
HIGH = 'HIGH24HOUR'
LOW = 'LOW24HOUR'
VOLUME = 'VOLUME24HOUR'
CHANGE = 'CHANGE24HOUR'
CHANGE_PERCENT = 'CHANGEPCT24HOUR'
MARKETCAP = 'MKTCAP'
NPERIODS = 100
TIMEFRAME = 'Day'
datetimeStringformat_to_csv = "%d-%m-%Y %H:%M"
data_directory = '/Users/ankitaggarwal/Applications/Cryptocurrency Trading System/crypto-analysis/data/'

In [4]:
# Defaults
CURR = 'BTC'
EXCHANGE = 'CCCAGG'
COIN = 'ETH'
COIN_LIST = ['BTC', 'ETH']
EXCHANGES = ['Bittrex','Binance']
EXCHANGES = ['Binance']

bittrex_exchange = ccxt.bittrex()
binance_exchange = ccxt.binance()
coinbasePro = ccxt.coinbasepro()
"""
kucoin_exchange = ccxt.kucoin()
huobiPro_exchange = ccxt.huobipro()
bitmex_exchange = ccxt.bitmex()
"""

list_of_exchanges = [coinbasePro]

In [21]:
#print (ccxt.exchanes)
key = vars(coinbasePro)
pprint.pprint(key)

{'aiohttpProxy': None,
 'aiohttpTrustEnv': False,
 'api': {'private': {'delete': ['orders', 'orders/{id}'],
                     'get': ['accounts',
                             'accounts/{id}',
                             'accounts/{id}/holds',
                             'accounts/{id}/ledger',
                             'accounts/{id}/transfers',
                             'coinbase-accounts',
                             'coinbase-accounts/{id}/addresses',
                             'fills',
                             'funding',
                             'fees',
                             'orders',
                             'orders/{id}',
                             'otc/orders',
                             'payment-methods',
                             'position',
                             'reports/{id}',
                             'users/self/trailing-volume'],
                     'post': ['conversions',
                              'deposits/coinbase-

In [6]:
pprint.pprint(vars(coinbasePro)['fees'])

{'funding': {'deposit': {'BCH': 0,
                         'BTC': 0,
                         'ETH': 0,
                         'EUR': 0.15,
                         'LTC': 0,
                         'USD': 10},
             'percentage': False,
             'tierBased': False,
             'withdraw': {'BCH': 0,
                          'BTC': 0,
                          'ETH': 0,
                          'EUR': 0.15,
                          'LTC': 0,
                          'USD': 25}},
 'trading': {'maker': 0.005,
             'percentage': True,
             'taker': 0.005,
             'tierBased': True}}


In [7]:
pprint.pprint(vars(binance_exchange).keys())

dict_keys(['precision', 'limits', 'exceptions', 'headers', 'balance', 'orderbooks', 'orders', 'tickers', 'trades', 'transactions', 'ohlcvs', 'currencies', 'options', 'decimal_to_precision', 'number_to_string', 'origin', 'userAgent', 'id', 'name', 'countries', 'rateLimit', 'certified', 'pro', 'has', 'timeframes', 'urls', 'api', 'fees', 'commonCurrencies', 'aiohttpProxy', 'aiohttpTrustEnv', 'arrayConcat', 'asyncioLoop', 'base16ToBinary', 'base58Alphabet', 'base58Decoder', 'base58Encoder', 'base58ToBinary', 'baseCurrencies', 'binaryConcat', 'binaryConcatArray', 'binaryToBase58', 'binaryToBase64', 'currenciesById', 'decimalToBytes', 'decimalToPrecision', 'deepExtend', 'encodeUriComponent', 'extractParams', 'filterBy', 'fromWei', 'groupBy', 'gzipDeflate', 'hasWeb3', 'implodeParams', 'inArray', 'indexBy', 'integerDivide', 'integerModulo', 'integerPow', 'isEmpty', 'isJsonEncodedObject', 'keyExists', 'lastHttpResponse', 'lastJsonResponse', 'lastResponseHeaders', 'loadedFees', 'marketsById', 'n

In [22]:
# For every exchange, fetch it's markets. Then depending on the JSON returned, prepare a list of coins for which historical data has to be downloaded.
def setupExchanges(list_of_exchanges):
    done = False
    i=0
    #df_markets = pd.DataFrame(markets)
    #bittrex_market = bittrex_exchange.fetchMarkets()
    #binance_market = binance_exchange.fetchMarkets()
    #kucoin_market = kucoin_exchange.fetchMarkets()
    #list_of_markets = [#bittrex_market,
                       #binance_market
     #                  kucoin_market #For kucoin the fetchMarkets function returns different dictionary keys
      #                  ]

    list_of_tuples = []
    for exchange in list_of_exchanges:
        coins_list = set()
        #if exchange.name == 'Cryptopia' or exchange.name == 'Bittrex' or exchange.name == 'Kucoin' or exchange.name == 'Huobi Pro':
            #continue #exchange.name == 'Binance' or 
        markets = exchange.fetchMarkets()
        pprint.pprint(markets[0])
        sys.exit(0)
        for row in markets:
            #row.base,row.quote,row.symbol,row.type,row.active
            #print(row.keys())
            tuple_data = (exchange.name,row['base'],row['quote'],row['symbol'],row['active'])
            list_of_tuples.append(tuple_data)
            
    return list_of_tuples

In [23]:
setupExchanges(list_of_exchanges)

{'active': True,
 'base': 'BTC',
 'baseId': 'BTC',
 'id': 'BTC-EUR',
 'info': {'base_currency': 'BTC',
          'base_increment': '0.00000001',
          'base_max_size': '200.00000000',
          'base_min_size': '0.00100000',
          'cancel_only': False,
          'display_name': 'BTC/EUR',
          'id': 'BTC-EUR',
          'limit_only': False,
          'margin_enabled': False,
          'max_market_funds': '600000',
          'min_market_funds': '10',
          'post_only': False,
          'quote_currency': 'EUR',
          'quote_increment': '0.01000000',
          'status': 'online',
          'status_message': '',
          'trading_disabled': False},
 'limits': {'amount': {'max': 200.0, 'min': 0.001},
            'cost': {'max': 600000.0, 'min': 10.0},
            'price': {'max': None, 'min': 0.01}},
 'maker': 0.005,
 'percentage': True,
 'precision': {'amount': 8, 'price': 2},
 'quote': 'EUR',
 'quoteId': 'EUR',
 'symbol': 'BTC/EUR',
 'taker': 0.005,
 'tierBased': Tru

SystemExit: 0

/Users/ankitaggarwal/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3275: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
